In [1]:
import pandas as pd
import json
import string

In [2]:
all_files_za = pd.read_csv("Classification/all_files_ZAv1.csv")

In [3]:
all_files_za.shape

(47886, 23)

In [4]:
new_artists_df = all_files_za[all_files_za["artistAppearanceCount"] == 0]

In [5]:
new_artists_df.shape

(913, 23)

In [6]:
new_artists_df.columns

Index(['trackAppearanceCount', 'artistAppearanceCount', 'rank', 'uri',
       'artist_names', 'track_name', 'source', 'peak_rank', 'previous_rank',
       'weeks_on_chart', 'streams', 'End Date', 'Week', 'Year', 'End Date Dt',
       'Seconds since Epoch', 'End Date Aggregated', 'ArtistCount',
       'main_artist', 'isTopTen', 'Artist and Track', 'rank difference',
       'Position over Time'],
      dtype='object')

In [7]:
new_artists_df_rel = new_artists_df[['trackAppearanceCount', "main_artist", 'End Date', 'track_name', 'source', 'streams', 'rank']]

##### Make the artist dictionary - position, track name, streams.

In [8]:
per_artist_df = new_artists_df[["main_artist", 'track_name', 'streams', 'rank']]

In [9]:
per_artist_df_idx = per_artist_df.set_index("main_artist")

In [10]:
per_artist_dict = per_artist_df_idx.to_dict('index')

In [11]:
# per_artist_dict

##### Make the general week ending dictionary

In [12]:
counts_artists_date_df = pd.DataFrame(new_artists_df_rel.groupby(["End Date"])["main_artist"].nunique()).reset_index()
counts_artists_date_df.columns = ['End Date', "count_main_artist"]

In [13]:
new_artists_df_rel

,trackAppearanceCount,main_artist,End Date,track_name,source,streams,rank
0,0,$NOT,2022-02-10,Doja,300 Entertainment,23789,194
1,0,$uicideboy$,2021-10-07,"...And to Those I Love, Thanks for Sticking Ar...",G59 Records,23341,182
2,0,031CHOPPA,2022-09-08,Usenemali,031Choppa,35968,128
3,0,2 Chainz,2018-06-21,Bigger Than You (feat. Drake & Quavo),Def Jam Recordings,5994,78
14,0,21 Savage,2018-03-15,Bank Account,"Slaughter Gang, LLC/Epic Records",1349,57
...,...,...,...,...,...,...,...
47862,0,pewdiepie,2018-12-13,Bitch Lasagna,Party In Backyard,5829,191
47863,0,rum.gold,2022-06-16,Call It What You Want,Leola LLC,29211,165
47871,0,salem ilese,2020-09-10,Mad at Disney,"Homemade Projects, LLC / TenThousand Projects",20005,107
47883,0,twocolors,2021-01-14,Lovefool,Virgin,16498,196


In [14]:
counts_artists_date_df["count_songs"] = pd.DataFrame(new_artists_df_rel.groupby(["End Date"])["track_name"].nunique()).reset_index()["track_name"]
counts_artists_date_df["count_source"] = pd.DataFrame(new_artists_df_rel.groupby(["End Date"])["source"].nunique()).reset_index()["source"]

In [15]:
counts_artists_date_df["sum_streams"] = pd.DataFrame(new_artists_df_rel.groupby(["End Date"])["streams"].agg("sum")).reset_index()["streams"]
counts_artists_date_df["min_rank"] = pd.DataFrame(new_artists_df_rel.groupby(["End Date"])["rank"].agg("min")).reset_index()["rank"]
counts_artists_date_df["max_rank"] = pd.DataFrame(new_artists_df_rel.groupby(["End Date"])["rank"].agg("max")).reset_index()["rank"]

In [16]:
unique_dts = new_artists_df_rel["End Date"].unique().tolist()
zeros = [""] * len(unique_dts)

dates_counts_dicts = dict(zip(unique_dts, zeros))

def makeartistsstring(dates_counts_dicts, x):
    dates_counts_dicts[x["End Date"]] = dates_counts_dicts[x["End Date"]] + ", " + x["main_artist"]
    
new_artists_df_rel.apply(lambda x : makeartistsstring(dates_counts_dicts, x), axis=1)

0        None
1        None
2        None
3        None
14       None
         ... 
47862    None
47863    None
47871    None
47883    None
47885    None
Length: 913, dtype: object

In [17]:
for a_dt in unique_dts:
    dates_counts_dicts[a_dt] = {"main_artists":dates_counts_dicts[a_dt]}

In [18]:
counts_artists_date_df_new = counts_artists_date_df.set_index('End Date')

In [19]:
dates_counts_dicts_else = counts_artists_date_df_new.to_dict('index')

In [20]:
# dates_counts_dicts_else

In [21]:
def mergeDictionary(dict_1, dict_2):
    """
        merge two dictionaries which are dictionaries of dictionaries 
        with the same main level keys but different keys sub-keys
    
    """
    dict_3 = {**dict_1, **dict_2}
    for key, value in dict_3.items():
        if key in dict_1 and key in dict_2:
            try:
                dict_3[key] = {k: v for d in [value , dict_1[key]] for k, v in d.items()}
            except:
                print("failed to unpack dict for this key", key)
                print("failed to unpack dict for this value", value)
#             
    return dict_3

In [22]:
fin_dict_script = mergeDictionary(dates_counts_dicts_else, dates_counts_dicts)

In [23]:
fin_dict_script['2018-03-15']

{'main_artists': ', 21 Savage, Avicii, Bad Wolves, Bazzi, Bebe Rexha, BlocBoy JB, Bruno Mars, Camila Cabello, Cashmere Cat, Clean Bandit, Coolio, DJ Khaled, David Guetta, Dr. Dre, Drake, Dua Lipa, Ed Sheeran, Eminem, Fort Minor, Future, G-Eazy, Hailee Steinfeld, Imagine Dragons, Jax Jones, Jay Rock, Jeremy Loops, Julia Michaels, Justin Timberlake, KALEO, Kanye West, Kendrick Lamar, Khalid, Kwesta, Liam Payne, Lil Pump, Lil Uzi Vert, Lil Wayne, Lil Xan, Lil Yachty, Logic, Maroon 5, Marshmello, Martin Garrix, Migos, NF, Offset, Portugal. The Man, Post Malone, R3HAB, Rich The Kid, Rob $tone, Rudimental, Sam Smith, Sean Paul, Selena Gomez, Shekhinah, The Chainsmokers, The Weeknd, The White Stripes, XXXTENTACION, Zedd',
 'count_main_artist': 61,
 'count_songs': 61,
 'count_source': 47,
 'sum_streams': 147241,
 'min_rank': 1,
 'max_rank': 86}

In [24]:
# Serializing json
json_object = json.dumps(fin_dict_script, indent=4)

with open("new_za_artists/all_dates.json", "w") as outfile:
        outfile.write(json_object)

In [25]:
# making the scripts for the different dates
all_dates_scripts = {}
for a_dt in unique_dts:  
    start = "For the week ending " + a_dt + "..."
    artists = "There were " + str(fin_dict_script[a_dt]["count_main_artist"]) + "... new artists making their debut on the chart"
    streams = "All in all, the new artists generated a total of " + str(fin_dict_script[a_dt]["sum_streams"]) + "... streams in their debut week"

    main_artists_ls = fin_dict_script[a_dt]["main_artists"].split(",")

    artist_listing = ""
    for artist in main_artists_ls:
        artist = artist.lstrip()
        artist = artist.rstrip()
        if(artist != ""):
            artist_listing = artist_listing + " At position " + str(per_artist_dict[artist]['rank']) + "...  we have " + artist + "... who is debuting with the song " + per_artist_dict[artist]['track_name'] + " with " + str(per_artist_dict[artist]['streams']) + " streams ... "
    
    week_complete_str = start + artists + artist_listing + streams
    all_dates_scripts[a_dt] = week_complete_str

In [26]:
# Serializing json
json_object = json.dumps(all_dates_scripts, indent=4)

with open("new_za_artists/all_dates_scripts.json", "w") as outfile:
        outfile.write(json_object)

In [27]:
# making the scripts for the different dates
for a_dt in unique_dts:  
    
    # Serializing json
    json_object = json.dumps(all_dates_scripts[a_dt], indent=4)
    
    new_artist_name = a_dt.translate(str.maketrans('', '', string.punctuation))

#     Writing to sample.json
    with open("new_za_artists/" + new_artist_name +".json", "w") as outfile:
        outfile.write(json_object)